---
title: Graphs
order: 5
---

This post implements (and improves) code in chapter 14 from the textbook Introduction to Computation and Programming Using Python, With Application to Computational Modeling and Understanding Data, third edition, by John V. Guttag, 2020. 

In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Tuple

In [2]:
@dataclass(frozen=True)
class Node:
    """Basic immutable graph node."""

    name: str

    def __str__(self) -> str:
        return self.name

Designing the inheritance pattern is difficult to me here. The substitution principle: A code that works on a superclass must work on a subclass. But if a code works on bidirectional edges (perhaps exploiting symmetry) it may not work on unidirectional edges (where the symmetry doesn't exist). So, if anything, bidirectional edge must inherit from unidirectional edge. But that doesn't work either, because unidirectional edge logically must have the sources and destinations, but bidirectional edge cannot have them. Which node should be returned by a bidirectional edge's "source" property? There is no clear answer, so I conclude bidirectional edge cannot inherit from unidirectional. Together these two facts mean neither can inherit from the other.

One solution would be to completely discard bidirectional edges and only work with directional ones - after all, we can build undirected graph with directed edges. But that would mean, while building an undirected graph, that we treat unidirectional edges as bidirectional, which is weird.

BaseEdge is a abstract base class for edges. An edge has two nodes, node1 and node2, can return them in a tuple (.nodes) and can be represented by a string where names of both nodes are separated by a separator. Also is hashable, which means it implements \_\_eq\_\_ and \_\_hash\_\_ methods.
_separator, \_\_eq\_\_ and \_\_hash\_\_ are abstract methods. Their implementation depends on whether the edge is bidirectional or unidirectional.

@ property means accessing `.nodes` on in instance will call this *method*, which works as a getter. It also blocks setting the attribute (via `.nodes = something`). But it doesn't block its mutation, were it e.g. a list.

In [ ]:
@dataclass(frozen=True)
class BaseEdge(ABC):
    """Abstract base class for immutable graph edge."""

    node1: Node
    node2: Node
    weight: int | float | None = None

    def __post_init__(self):
        # (if weighted, then weight not None)
        if self.weighted and self.weight < 0:  # type: ignore
            raise ValueError("Weight cannot be negative")

    @property
    def weighted(self) -> bool:
        return self.weight is not None

    @property
    def nodes(self) -> Tuple[Node, Node]:
        return self.node1, self.node2

    @property
    @abstractmethod
    def _separator(self) -> str:
        pass

    def __str__(self) -> str:
        node1, node2 = self.nodes
        sep = self._separator
        half = len(sep) // 2
        if self.weighted:
            return f"{node1} {sep[:half]}({self.weight}){sep[half:]} {node2}"
        else:
            return f"{node1} {self._separator} {node2}"


@dataclass(frozen=True)
class Edge(BaseEdge):
    """Bidirectional immutable graph edge."""

    def __init__(self, node1: Node, node2: Node, weight: int | float | None = None):
        # Canonical ordering for dataclass's __eq__ and __hash__
        # to work correctly and for better (consistent) str representation
        if node1.name > node2.name:
            BaseEdge.__init__(self, node2, node1, weight)
        else:
            BaseEdge.__init__(self, node1, node2, weight)

    @property
    def _separator(self) -> str:
        return "<---->"


@dataclass(frozen=True)
class DirectedEdge(BaseEdge):
    """Unidirectional immutable graph edge."""

    def __init__(self, src: Node, dest: Node, weight: int | float | None = None):
        # Improves signature of the constructor
        # Invariant: src=node1 and dest=node2
        BaseEdge.__init__(self, src, dest, weight)

    @property
    def src(self) -> Node:
        return self.nodes[0]

    @property
    def dest(self) -> Node:
        return self.nodes[1]

    @property
    def _separator(self) -> str:
        return "----->"

There is one unfortunate thing about this implementation: `_separator` isn't part of the dataclass machinery (which means it isn't part of `repr`, `eq`, `hash` etc.). It doesn't matter to the functionality, because both `node1` and `node2` are completely determined by either `src` and `dest` or by the name of the two nodes. But the implication is that first, `repr` isn't connected to how the object is created, which hurts readibility and understandability. And second, just seeing the dataclass parameters of an object (node1=..., node2=..., weight=...) isn't enough to determine if the edge is directed or undirected.

### Testing of Nodes and Edges

In [15]:
node1 = Node("1")
node2 = Node("2")

# BaseEdge()  # TypeError: Can't instantiate abstract class BaseEdge without an implementation for abstract method '_separator'

e12 = Edge(node1, node2)
we12_314 = Edge(node1, node2, 3.14)
dir12 = DirectedEdge(node1, node2)
wdir12_314 = DirectedEdge(node1, node2, 3.14)

e21 = Edge(node2, node1)
we21_314 = Edge(node2, node1, 3.14)
dir21 = DirectedEdge(node2, node1)
wdir21_314 = DirectedEdge(node2, node1, 3.14)

we12_3 = Edge(node1, node2, 3)
wdir12_3 = DirectedEdge(node1, node2, 3)

In [16]:
print(f"{e12 == e21}: {e12} equals {e21}")
print(f"{we12_314 == we21_314}: {we12_314} equals {we21_314}")
print(f"{dir12 == dir21}: {dir12} equals {dir21}")
print(f"{wdir12_314 == wdir21_314}: {wdir12_314} equals {wdir21_314}")
print(f"{we12_314 == we12_3}: {we12_314} equals {we12_3}")
print(f"{wdir12_314 == wdir12_3}: {wdir12_314} equals {wdir12_3}")

True: 1 <----> 2 equals 1 <----> 2
True: 1 <--(3.14)--> 2 equals 1 <--(3.14)--> 2
False: 1 -----> 2 equals 2 -----> 1
False: 1 ---(3.14)--> 2 equals 2 ---(3.14)--> 1
False: 1 <--(3.14)--> 2 equals 1 <--(3)--> 2
False: 1 ---(3.14)--> 2 equals 1 ---(3)--> 2


In [17]:
print({e12, e21})  # exactly what we wanted because they are the same edge!
print({we12_314, we21_314})
print({node1, Node("1"), node2})
DirectedEdge(node1, node2)  # unfortunately the improved signature not part of repr

{Edge(node1=Node(name='1'), node2=Node(name='2'), weight=None)}
{Edge(node1=Node(name='1'), node2=Node(name='2'), weight=3.14)}
{Node(name='1'), Node(name='2')}


DirectedEdge(node1=Node(name='1'), node2=Node(name='2'), weight=None)

In [18]:
try:
    edge_neg = Edge(node1, node2, -3.14)
except ValueError:
    print("test passed")

test passed


In [ ]:
class Digraph:
    """Directed graph."""

    # nodes is a list of the nodes in the graph.
    # edges is a dict mapping each node to a list of its children implemented
    # as tuples, where the first element is the child node and the second
    # element, possibly empty (depending on weighted), is the weight of the edge.
    # The first added edge (or the argument to __init__) determines whether
    # the Digraph is weighted for the rest of its life.
    # Internally, an "edge" is one-way only.
    # Bidirectional edges treated as having an "edge" in both directions.

    def __init__(self, weighted=None):
        """When weighted is None, it is determined by the first added edge."""
        self._nodes = []
        self._edges = {}
        self.weighted = weighted
        self._last_computed_str = "Empty graph"
        self._str_valid = True

    def add_node(self, node) -> None:
        if self.has_node(node):
            raise ValueError("Duplicate node")
        else:
            self._nodes.append(node)
            self._edges[node] = []
        self._str_valid = False

    def _insert_edge(self, src, dest, weight) -> None:
        if dest in self.children_of(src):
            raise ValueError("Duplicate edge")
        if self.weighted:
            self._edges[src].append((dest, weight))
        else:
            self._edges[src].append((dest,))
        self._str_valid = False

    def _check_weighted(self, edge) -> None:
        if edge.weighted and not self.weighted:
            raise ValueError("This graph is unweighted, cannot add weighted edge")
        if self.weighted and not edge.weighted:
            raise ValueError("This graph is weighted, cannot add unweighted edge")

    def add_directed_edge(self, edge) -> None:
        node1, node2 = edge.nodes
        self._check_node_exists(node1)
        self._check_node_exists(node2)
        if self.weighted is not None:
            self._check_weighted(edge)
        else:
            self.weighted = edge.weighted
        self._insert_edge(edge.src, edge.dest, edge.weight)

    def add_undirected_edge(self, edge) -> None:
        """Treated as two directed edges."""
        self.add_directed_edge(edge)
        # both nodes and weighted now checked
        # using invariant of type Edge
        self._insert_edge(edge.node2, edge.node1, edge.weight)

    def children_of(self, node) -> list:
        # never returns the list itself, just a copy
        self._check_node_exists(node)
        return [tup[0] for tup in self._edges[node]]

    def has_node(self, node) -> bool:
        return node in self._nodes

    def _check_node_exists(self, node) -> None:
        if not self.has_node(node):
            raise ValueError("Node not in graph")

    def __str__(self) -> str:
        res = ""
        no_edges = []
        for node in self._nodes:
            if len(self.children_of(node)) == 0:
                no_edges.append(node)
                continue
            for child in self._edges[node]:
                if self.weighted:
                    res += f"{node} ---({child[1]})--> {child[0]}\n"
                else:
                    res += f"{node} -----> {child[0]}\n"
        if len(no_edges) == 0:
            res += "No nodes without edges."
        else:
            res += "Nodes without edges:\n"
            for node in no_edges:
                res += f"{node}\n"
            res = res[:-1]
        self._last_computed_str = res
        self._str_valid = True
        return res


class Graph(Digraph):
    def add_directed_edge(self, edge):
        raise TypeError("Cannot add directed edge into undirected graph")

    def __str__(self) -> str:
        res = ""
        no_edges = []
        to_print = []
        for node in self._nodes:
            if len(self.children_of(node)) == 0:
                no_edges.append(node)
                continue
            for child in self._edges[node]:
                if self.weighted:
                    # conveniently handles the cannonical ordering and the *in* operator
                    edge = Edge(node, child[0], child[1])
                else:
                    edge = Edge(node, child[0])
                if edge not in to_print:
                    to_print.append(edge)
        for edge in to_print:
            res += f"{edge}\n"
        if len(no_edges) == 0:
            res += "No nodes without edges."
        else:
            res += "Nodes without edges:\n"
            for node in no_edges:
                res += f"{node}\n"
            res = res[:-1]
        self._last_computed_str = res
        self._str_valid = True
        return res